# Generating the Blind Set

### 1. Downloading Data from the PDB

Checked the boxes "Sequence", "Entity Polymer Type", "Chain ID", "Entity ID", "Entry ID (Polymer Entity Identifyers). 

[here the link to my search](https://www.rcsb.org/search?request=%7B%22query%22%3A%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22greater%22%2C%22negation%22%3Afalse%2C%22value%22%3A%222015-01-31T00%3A00%3A00Z%22%2C%22attribute%22%3A%22rcsb_accession_info.deposit_date%22%7D%2C%22node_id%22%3A0%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22exact_match%22%2C%22negation%22%3Afalse%2C%22value%22%3A%22X-RAY%20DIFFRACTION%22%2C%22attribute%22%3A%22exptl.method%22%7D%2C%22node_id%22%3A1%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22less_or_equal%22%2C%22negation%22%3Afalse%2C%22value%22%3A2.5%2C%22attribute%22%3A%22rcsb_entry_info.resolution_combined%22%7D%2C%22node_id%22%3A2%7D%2C%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22or%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22exact_match%22%2C%22negation%22%3Afalse%2C%22value%22%3A%22Protein%20(only)%22%2C%22attribute%22%3A%22rcsb_entry_info.selected_polymer_entity_types%22%7D%2C%22node_id%22%3A3%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22exact_match%22%2C%22negation%22%3Afalse%2C%22value%22%3A%22Protein%2FNA%22%2C%22attribute%22%3A%22rcsb_entry_info.selected_polymer_entity_types%22%7D%2C%22node_id%22%3A4%7D%5D%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22range_closed%22%2C%22negation%22%3Afalse%2C%22value%22%3A%5B50%2C300%5D%2C%22attribute%22%3A%22entity_poly.rcsb_sample_sequence_length%22%7D%2C%22node_id%22%3A5%7D%5D%7D%5D%2C%22label%22%3A%22text%22%7D%5D%2C%22label%22%3A%22query-builder%22%7D%2C%22return_type%22%3A%22entry%22%2C%22request_options%22%3A%7B%22pager%22%3A%7B%22start%22%3A0%2C%22rows%22%3A100%7D%2C%22scoring_strategy%22%3A%22combined%22%2C%22sort%22%3A%5B%7B%22sort_by%22%3A%22score%22%2C%22direction%22%3A%22desc%22%7D%5D%7D%2C%22request_info%22%3A%7B%22src%22%3A%22ui%22%2C%22query_id%22%3A%22dc19df09287d4c5a80018000a03e2a6d%22%7D%7D)

* Downloaded as CSV 

For some reason it automatically adds "Entry ID" as column 1 --> removed it using awk:

In [1]:
head -2 1.csv

"Entry ID","Sequence","Chain ID","Entry Id (Polymer Entity Identifiers)",
"5EV3","GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN","A","5EV3",


In [2]:
cat 1.csv | awk '{sub(/[^,]*/,"");sub(/,/,"")} 1' > 1new.csv 
cat 2.csv | awk '{sub(/[^,]*/,"");sub(/,/,"")} 1' > 2new.csv
cat 3.csv | awk '{sub(/[^,]*/,"");sub(/,/,"")} 1' > 3new.csv
cat 4.csv | awk '{sub(/[^,]*/,"");sub(/,/,"")} 1' > 4new.csv 
cat 5.csv | awk '{sub(/[^,]*/,"");sub(/,/,"")} 1' > 5new.csv 

In [5]:
head -5 1new.csv #now it looks like this

"Sequence","Chain ID","Entry Id (Polymer Entity Identifiers)",
"GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN","A","5EV3",
"UUUUUUUU","B","5EV3",
"GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN","A","5EV2",
"UUUUUUUU","B","5EV2",


### 2. Generating the FASTA

I want to consider the comma as a field sepparator but since there are several chains per entry denoted as e.g.:

``` "CAGTTTCAAACTC","D, I","5FD3",```

I need to remove the comma between the chains first and replace it with a space:
*  ``` sed 's/\, / /g' $i.csv ```

Then pipe it to awk and define the comma as field sepparator

* ``` awk -F ',' 'length($1) > 50 {print ">"$3":" substr($2,1,2) "\n"$1}' ```


In [8]:
for i in {1..5}
do
    sed 's/\, / /g' ${i}new.csv | awk -F ',' 'length($1) > 50 {print ">"$3":" substr($2,1,2) "\n"$1}' | sed 's/\"//g'  > nocomma${i}.fasta
done

In [14]:
head nocomma1.fasta

>5EV3:A
GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN
>5EV2:A
GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN
>5EV4:A
GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN
>5F8A:A
SLRSDLINALYDENQKYDVCGIISAEGKIYPLGSDTKVLSTIFELFSRPIINKIAEKHGYIVEEPKQQNHYPDFTLYKPSEPNKKIAIDIKTTYTNKENEKIKFTLGGYTSFIRNNTKNIVYPFDQYIAHWIIGYVYTRVATRKSSLKTYNINELNEIPKPYKGVKVFLQDKWVIAGDLAGSGNTTNIGSIHAHYKDFVEGKGIFDSEDEFLDYWRNYERTSQLRNDKYNNISEYRNWIYRGRK
>5FMP:A
MAVLAESELGSEAQRERRKRILDATMAIASKGGYEAVQMRAVADRADVAVGTLYRYFPSKVHLLVSALGREFSRIDAKTDRSAVAGATPFQRLNFMVGKLNRAMQRNPL

In [16]:
grep ">" nocomma1.fasta | wc -l

    6604
